# A 10-minute Mean-Reversion Template

This simple template provides basic instructions for helping you within 10 minutes to develop and submit a mean-reversion strategy on liquid assets. Mean reversion in finance relies on the belief that stock prices tend to move to their average over time.

In [ ]:
# Import basic libraries for manipulating data.

# Please refer to xarray.pydata.org for xarray documentation.

# xarray works optimally with N-dimensional datasets in Python
# and is well suited for financial datasets with labels "time",
# "field" and "asset". xarray data structures can also be easily
# converted to pandas dataframes.

import xarray as xr

import numpy as np
import pandas as pd

# Import quantnet libraries.

import qnt.data as qndata       # data loading and manipulation
import qnt.stepper as qnstepper # strategy definition
import qnt.stats as qnstats     # key statistics
import qnt.graph as qngraph     # graphical tools

# display function for fancy displaying:
from IPython.display import display

In [ ]:
# Load all available asset names since given date.

# Note that submissions will be accepted only with a 3-year
# simulated track record on historical data before submission
# date.

assets = qndata.load_assets(min_date="2015-01-01")

assets_names = [i["id"] for i in assets]

# Load all available data since given date.

# It is possible to set a max_date in the call in order to
# develop the system on a limited in-sample period and later
# test the system on unseen data after max_date.

# A submission will be accepted only if no max_date is set,
# as submissions will be evaluated on live data on a daily basis.

data = qndata.load_data(min_date="2015-01-01",
                       #max_date="2019-02-28",
                        dims=("time", "field", "asset"),
                        assets=assets_names)

In [ ]:
# A simple mean-reversion strategy on liquid assets allocates
# fractions of capital according to the deviation from the historical
# average price.
# Here xarray data structures are converted to pandas
# dataframes for simplicity in order to describe the
# development process.

# xarray.DataArray are converted to pandas dataframes
# and time stamps are arranged in ascending order:
is_liquid = data.loc[:, "is_liquid", :].to_pandas().iloc[::-1,:]

# extract close prices:
prices = data.loc[:, "close", :].to_pandas().iloc[::-1,:]

# compute average:
avg_prices = prices.rolling(100).mean()

# compute size of relative returns and take average:
returns = prices.pct_change(1).abs().rolling(10).mean()

# go long when prices are too low compared to the mean:
go_long = (prices < avg_prices * (1.0-returns)).astype(int)

# go short when prices are too high compared to the mean:
go_short = (prices > avg_prices * (1.0+returns)).astype(int)

# select liquid assets only:
liq_avg_returns = (is_liquid > 0.0).astype(int).multiply(go_long-go_short)

# set and normalize weights according to past performance:
weights = liq_avg_returns.div(liq_avg_returns.abs().sum(axis=1, skipna=True), axis=0)
weights = weights.fillna(0.0)

display(weights)

In [ ]:
# check that we are fully invested <=> sum of abs(weights) = 1:
display(weights.abs().sum(axis=1))

In [ ]:
# visualize positions for a selected asset:
aapl_frac = weights["NASDAQ:AAPL"]
qngraph.make_plot(aapl_frac.index, aapl_frac, name= "aapl")

In [ ]:
# Using the following function ensures that no forward-looking
# is taking place, at the cost of a slower execution. In the
# step function we work directly with xarray data structures
# avoiding conversion to pandas dataframes for avoiding a loss
# of computational speed.

def step(data):
    """Implements the previous strategy and evaluates
    positions at a fixed point in time.

    Args: data, an xarray.DataArray with 3 coordinates:
          time, field, asset

    Returns: an xarray.DataArray with fractions which
        will be allocated to all assets on the next day
    """

    is_liquid = data.loc[::-1,"is_liquid",:]
    prices = data.loc[::-1,"close",:]
    avg_prices = prices.rolling({"time":100}).mean()
    prices_shifted = prices.shift({"time":1})
    returns = (abs((prices - prices_shifted)/prices_shifted).rolling({"time":10}).mean())
    go_long = (prices < avg_prices * (1.0-returns)).astype(int)
    go_short = (prices > avg_prices * (1.0+returns)).astype(int)
    liq_avg_returns = (is_liquid > 0.0).astype(int)*(go_long-go_short)
    weights = liq_avg_returns / abs(liq_avg_returns).sum("asset")
    weights = weights.fillna(0.0)

    return weights[-1]

In [ ]:
# a single call to the step function generates positions
# for the last available date in the simulation timeframe:
display(step(data))

In [ ]:
# The quantnet libraries provide a test function for checking
# formal correctness of the defined strategy. Input arguments
# are the data, the step function and a warm-up period, which
# we set in this case to 100 trading days.

output = qnstepper.test_strategy(data, step=step, init_data_length=100)

In [ ]:
# Calculate statistics on a rolling basis.

# Transactions are punished with slippage equal to a given
# fraction of ATR14 (read more about slippage in our full
# Strategy Buy and Hold template). We evaluate submissions
# using 5% of ATR14 for slippage.

# Mean return, volatility and Sharpe ratio are computed on a
# rolling basis using a lookback period of 3 years.

stat = qnstats.calc_stat(data, output, slippage_factor=0.05)

display(stat.to_pandas().tail())

In [ ]:
def print_stat(stat):
    """Prints selected statistical key indicators:
       - the global Sharpe ratio of the strategy;
       - the global mean profit;
       - the global volatility;
       - the maximum drawdown.

       Note that Sharpe ratio, mean profit and volatility
       apply to  max simulation period, and not to the
       rolling basis of 3 years.
    """

    days = len(stat.coords["time"])
    
    returns = stat.loc[:, "relative_return"]

    equity = stat.loc[:, "equity"]
    
    sharpe_ratio = qnstats.calc_sharpe_ratio_annualized(
        returns,
        max_periods=days,
        min_periods=days).to_pandas().values[-1]

    profit = (qnstats.calc_mean_return_annualized(
        returns,
        max_periods=days,
        min_periods=days).to_pandas().values[-1])*100.0

    volatility = (qnstats.calc_volatility_annualized(
        returns,
        max_periods=days,
        min_periods=days).to_pandas().values[-1])*100.0

    max_ddown = (qnstats.calc_max_drawdown(
        qnstats.calc_underwater(equity)).to_pandas().values[-1])*100.0

    print("Sharpe Ratio         : ", "{0:.3f}".format(sharpe_ratio))
    print("Mean Return [%]      : ", "{0:.3f}".format(profit))
    print("Volatility [%]       : ", "{0:.3f}".format(volatility))
    print("Maximum Drawdown [%] : ", "{0:.3f}".format(-max_ddown))

print_stat(stat)

In [ ]:
# show plot with profit and losses:
performance = stat.to_pandas()["equity"]
qngraph.make_plot_filled(performance.index, performance, name="PnL (Equity)", type="log")

In [ ]:
# show underwater chart:
UWchart = stat.to_pandas()["underwater"]
qngraph.make_plot_filled(UWchart.index, UWchart, color="darkred", name="Underwater Chart")

In [ ]:
# show rolling Sharpe ratio on a 3-year basis:
SRchart = stat.to_pandas()["sharpe_ratio"].iloc[20:]
qngraph.make_plot_filled(SRchart.index, SRchart, color="#F442C5", name="Rolling SR")

In [ ]:
# show bias chart:
biaschart = stat.to_pandas()["bias"]
qngraph.make_plot_filled(biaschart.index, biaschart, color="#5A6351", name="Bias Chart")

In [ ]:
# Finally, we write the last mandatory step for submission,
# namely writing output to file:

qndata.write_output(output)

At this stage code is ready for submission. Just click on the submission button in your account page and we will evaluate your strategy live on our servers!

For speeding up evaluation, you can consider submitting a copy with only relevant steps and excluding plots. Your code in the final notebook you submit can be as simple as the following, which we include into the Mean-Reversion Submission Raw Template:

In [ ]:
"""
import xarray as xr
import qnt.data as qndata

data = qndata.load_data(min_date="2015-01-01",
                        dims=("time", "field", "asset"))

is_liquid = data.loc[::-1,"is_liquid",:]

prices = data.loc[::-1,"close",:]
avg_prices = prices.rolling({"time":100}).mean()

prices_shifted = prices.shift({"time":1})
returns = (abs((prices - prices_shifted)/prices_shifted).rolling({"time":10}).mean())

go_long = (prices < avg_prices * (1.0-returns)).astype(int)
go_short = (prices > avg_prices * (1.0+returns)).astype(int)

liq_avg_returns = (is_liquid > 0.0).astype(int)*(go_long-go_short)

weights = liq_avg_returns / abs(liq_avg_returns).sum("asset")
weights = weights.fillna(0.0)

qndata.write_output(weights)
"""